# Modèle de Speech-to-Text pour le Fon

Ce notebook propose une base de départ pour construire un système de reconnaissance vocale pour le Fon.

**Objectifs :**
 - Préparer un dataset de Fon (audio et transcription).
 - Extraire des caractéristiques audio, en intégrant éventuellement des informations sur le pitch pour capturer l’intonation.
 - Utiliser un modèle pré-entraîné (whisper).
 - Fine-tuner le modèle sur le dataset.



In [ ]:
# Chargement de mon Drive
from google.colab import drive
drive.mount('/content/drive')

# Importation des packages

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
DATASET_DIR = Path("/content/drive/MyDrive/dataset")
TRAIN_AUDIO_DIR = DATASET_DIR / "train"
VALIDATION_AUDIO_DIR = DATASET_DIR / "valid"

TRAIN_CSV_PATH = DATASET_DIR / "train.csv"
VALIDATION_CSV_PATH = DATASET_DIR / "valid.csv"

In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
valid_df = pd.read_csv(VALIDATION_CSV_PATH)

train_df.info()
train_df.sample(10)

# Chargement et préparation du dataset

In [ ]:
!pip -q install datasets

In [ ]:
import os
from datasets import Dataset, Audio

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

train_dataset = train_dataset.train_test_split(test_size=0.95)["train"] # Pour éviter crash de session dû à la RAM
valid_dataset = valid_dataset.train_test_split(test_size=0.95)["train"]


# On ajoute le chemin complet vers les fichiers audio
def add_audio_path(DIR, row):
    row["audio"] = os.path.join(str(DIR), row["filename"])
    return row

train_dataset = train_dataset.map(lambda row: add_audio_path(TRAIN_AUDIO_DIR, row))
valid_dataset = valid_dataset.map(lambda row: add_audio_path(VALIDATION_AUDIO_DIR, row))

train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
valid_dataset = valid_dataset.cast_column("audio", Audio(sampling_rate=16000))

# Préparation du modèle et du processor (Whisper)

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_name = "openai/whisper-tiny"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Prétraitement des données

In [ ]:
def preprocess_function(batch):
    audio_arrays = [x["array"] for x in batch["audio"]]

    inputs = processor.feature_extractor(audio_arrays, sampling_rate=16000, return_tensors="np") # Extraction des features audio

    texts = ["translate: " + txt for txt in batch["utterance"]] # On ajoute "translate : " devant chaque texte
    labels = processor.tokenizer(texts, return_tensors="np").input_ids

    batch["input_features"] = inputs.input_features
    batch["labels"] = labels
    return batch

# Prétraitement des données
dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Datacollector personnalisé

In [ ]:
from dataclasses import dataclass
import torch

@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: WhisperProcessor

    def __call__(self, features):
        # Regroupement et padding des input_features
        input_features = [f["input_features"] for f in features]
        batch_input = self.processor.feature_extractor.pad(
            {"input_features": input_features}, return_tensors="pt"
        )

        # Regroupement et padding des labels
        labels = [f["labels"] for f in features]
        batch_labels = self.processor.tokenizer.pad(
            {"input_ids": labels}, return_tensors="pt"
        )

        batch_labels["input_ids"][batch_labels["input_ids"] == self.processor.tokenizer.pad_token_id] = -100 # padding token par -100 pour les éviter dans la loss

        batch = {
            "input_features": batch_input["input_features"],
            "labels": batch_labels["input_ids"]
        }

        return batch

data_collator = DataCollatorSpeechSeq2Seq(processor=processor)

# Métriques

In [ ]:
!pip -q install evaluate sacrebleu

In [ ]:
import evaluate
import numpy as np

bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    """
    Cette fonction reçoit les prédictions et les labels sous forme de tuples,
    décode les séquences et calcule le score BLEU.
    """
    pred_ids, label_ids = eval_preds

    # Décodage des prédictions en textes
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    # Remplace les tokens -100 par le token de padding pour le décodage des labels
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Le format attendu pour sacreBLEU : une liste de références par prédiction (ici, une seule référence par prédiction)
    references = [[ref] for ref in label_str]

    # Calcul du score BLEU
    result = bleu_metric.compute(predictions=pred_str, references=references)
    return {"bleu": result["score"]}

# Configuration de l'env d'entrainement

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

NUM_EPOCHS = 3

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-fon-fr",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    num_train_epochs=NUM_EPOCHS,
    eval_strategy="epoch",
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    logging_steps=NUM_EPOCHS%5,
    push_to_hub=False,
    run_name="Fongbe_Experiment"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)

# Config de wandb

In [ ]:
!pip install -q wandb

In [ ]:
import wandb

wandb.login(key="")

wandb.init(project="IWSLT_Fongbe", name="exp1_baseline")

# Entrainement du modèle

In [ ]:
trainer.train()

# Test sur un échantillon de l'ensemble de validation

In [ ]:
results = trainer.evaluate()
print("Résultats sur l'ensemble de validation :", results)

In [ ]:
import torch

sample = dataset["validation"][0]

correct_translate = sample["utterance"]

# On prépare les input_features au format tensor
input_features = torch.tensor(sample["input_features"]).unsqueeze(0)

# Génération de la traduction à partir des input_features
generated_ids = model.generate(input_features, max_length=100)
translation = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Traduction attendue : ", correct_translate)
print("Traduction générée : ", translation)